# Examlpe Code

In [2]:
from CaptioningModel import *
import time
import multiprocessing
import pymp
from joblib import Parallel, delayed
import ray

model = CaptioningModel()
cpus = 2
images = 4

@ray.remote
def ray_predict(model, image_index):
    return model.predict(model.train_images[image_index])

if __name__=="__main__":
    print('Pymp')
    
    with pymp.Parallel(num_threads=cpus) as p:
        start = time.time()
        result = [model.predict(model.train_images[i]) for i in p.xrange(images)]
        end = time.time()
        
    print(end - start)
    print()
    
    print('Serial')
    
    start = time.time()
    result = [model.predict(model.train_images[i]) for i in range(images)]
    end = time.time()
    
    print(end - start)
    print()
    
    print('Multiprocessing')
    
    with multiprocessing.Pool(processes=cpus) as pool:
        start = time.time()
        result = pool.map(model.predict, model.train_images[:images])
        end = time.time()
    
    print(end - start)
    print()
    
    print('Joblib')
    
    start = time.time()
    result = Parallel(n_jobs=cpus)(delayed(model.predict)(model.train_images[i]) for i in range(images))
    end = time.time()
    
    print(end - start)
    print()
    
    print('Ray')

    ray.init(num_cpus=cpus)
    
    start = time.time()
    result = ray.get([ray_predict.remote(model, i) for i in range(images)])
    end = time.time()
    
    ray.shutdown()
    
    print(end - start)
    print()

Pymp


KeyboardInterrupt: 

In [3]:
from CaptioningModel import *
import time
import multiprocessing
import pymp
from joblib import Parallel, delayed
import ray
import matplotlib.pyplot as plt

cpus = 2
images = 50
model = CaptioningModel()
cores = [i+1 for i in range(12)]

@ray.remote
def ray_predict(model, image_index):
    return model.predict(model.train_images[image_index])

In [ ]:
with pymp.Parallel(num_threads=cpus) as p:
    start = time.time()
    result = [model.predict(model.train_images[i]) for i in p.range(images)]
    end = time.time()

print(end - start)

In [3]:
start = time.time()
result = [model.predict(model.train_images[i]) for i in range(images)]
end = time.time()

print(end - start)

17.604115962982178


In [6]:
if __name__=="__main__":
    with multiprocessing.Pool(processes=cpus) as pool:
        start = time.time()
        result = pool.map(model.predict, model.train_images[:images])
        end = time.time()
    
    print(end - start)

12.09216022491455


In [ ]:
times = []

for item in cores:
    cpus = item
    
    start = time.time()
    result = Parallel(n_jobs=cpus)(delayed(model.predict)(model.train_images[i]) for i in range(images))
    end = time.time()

    times.append(end - start)
    print(end - start)
    
plt.plot(cores, times)
plt.show()

In [6]:
ray.init(num_cpus=cpus)
    
start = time.time()
result = ray.get([ray_predict.remote(model, i) for i in range(images)])
end = time.time()

ray.shutdown()

print(end - start)

2021-04-18 12:20:39,946	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
(pid=60255) 2021-04-18 12:20:44.628698: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
(pid=60255) 2021-04-18 12:20:44.628881: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(pid=60255) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=60256) 2021-04-18 12:20:44.628807: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
(pid=60256) 2021-04-18 12:20:44.628964: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical o

12.734359979629517


# Example Backend Code
# Import

In [1]:
import pickle
import numpy as np
from itertools import groupby
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input, InceptionV3

# Directories

In [2]:
train_images_dir = 'Flickr_Data/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt'
test_images_dir = 'Flickr_Data/Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt'
images_dir = 'Flickr_Data/Flickr_Data/Images/'
model_dir = 'data/model_199.h5'
wordtoix_dir = 'data/wordtoix.pkl'
ixtoword_dir = 'data/ixtoword.pkl'

# Train & Test images

In [3]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [4]:
train_images = list()
doc = load_doc(train_images_dir)

for line in doc.split('\n'):
    if len(line) < 1:
        continue
    
    identifier = line.split('.')[0]
    train_images.append(images_dir + identifier + '.jpg')

print('Dataset: %d' % len(train_images))

Dataset: 6000


In [5]:
test_images = list()
doc = load_doc(test_images_dir)

for line in doc.split('\n'):
    if len(line) < 1:
        continue
    
    identifier = line.split('.')[0]
    test_images.append(images_dir + identifier + '.jpg')

print('Dataset: %d' % len(test_images))

Dataset: 1000


# Prediction Functions

In [6]:
start_token = 'Start '
end_token = ' End'
max_length = 30
wordtoix = pickle.load(open(wordtoix_dir, 'rb'))
ixtoword = pickle.load(open(ixtoword_dir, 'rb'))

In [7]:
def preprocess_image(filename):
    model = InceptionV3(weights='imagenet')
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    image = load_img(filename, target_size=(299, 299))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    feature = model.predict(image, verbose=0)
    return np.reshape(feature, (1, feature.shape[1]))

In [8]:
def greedySearch(photo, model):
    in_text = start_token[:-1]
    
    for i in range(15):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence])
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        
        if word == end_token[1:]:
            break
    
    final = in_text.split()
    final = final[1:-1]
    final = [x[0] for x in groupby(final)]
    final = ' '.join(final)
    return final

# Load Model

In [9]:
model = load_model(model_dir)

In [10]:
def predict(image_dir):
    global model
    image = preprocess_image(image_dir)
    return greedySearch(image, model)

# Predict Caption

In [11]:
caption = predict(train_images[2817])
print(caption)

కుక్కలతో బీచ్ లో ముగ్గురు కుక్కలతో ఆడుతున్నారు చూస్తున్నారు బంతిని చూస్తున్నారు మరొకటి కుక్క గాలిలో దూకుతోంది
